In [1]:
import pandas as pd
import nltk

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import pickle

import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = pd.read_csv("../data/dataset/dataset.csv", sep=';', encoding="ISO-8859-1")

In [3]:
dataset.head()

,Tweet,label
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",baterai negatif
1,Kmera iphone 11 mmeng beda,kamera positif
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera negatif
3,Apple meluncurkan iPhone 11 dengan peningkatan...,baterai positif
4,"xs max, senderrr. tapi rugi sih kalo kamu beli...",spesifikasi positif


## Clean Data

1. Menghilangkan simbol

In [4]:
def clean(data):
    data['Tweet_Parsed_1'] = data['Tweet'].str.replace("\\\\r", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("\\\\n", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("\n", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("\r", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("&amp", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("&gt", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("&lt", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("    ", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace(r'''(?i)\b((?:https|http?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace('"', '')
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("\\\\x[a-zA-z0-9][a-zA-z0-9]","")
    return data

In [5]:
dataset = clean(dataset)

2. Lowercase

In [6]:
def lowercase(data):
    data['Tweet_Parsed_2'] = data['Tweet_Parsed_1'].str.lower()
    return data

In [7]:
dataset = lowercase(dataset)

3. Menghilangkan tanda baca

In [8]:
def removepunct(data):
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_2']
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("'s","")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("can't","can not")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("ain't","am not")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("n't"," not")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("'re"," are")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace('[^a-zA-Z0-9]+',' ')
    return data

In [9]:
dataset = removepunct(dataset)

## Normalization

In [10]:
normal = pd.read_csv("../data/normalisasi/kamusnormalisasi.csv", sep=",", encoding='latin-1',header=None,names=["non-standar","standar"])

In [11]:
normal

,non-standar,standar
0,kmera,kamera
1,dijangka,dalam
2,dimuatkan,dimasukan
3,kapasiti,kapasitas
4,bateri,baterai
...,...,...
282,mmg,memang
283,software,perangkat lunak
284,say,mengatakan
285,bangettt,sekali


In [12]:
nonstandar = normal['non-standar'].values.tolist()
standar = normal['standar'].values.tolist()

In [13]:
def normalizeText(text, st_, nst_):
#     text = text.split(" ") 
    text = str(text).split(" ")
    for i in range(len(text)):
        if text[i] in nst_:
            index = nst_.index(text[i])
            text[i] = st_[index]
    return ' '.join(map(str, text))

def normalizeData(data, st_, nst_):
    data['Tweet_Parsed_4'] = data['Tweet_Parsed_3']
    data['Tweet_Parsed_4'] = data['Tweet_Parsed_4'].map(lambda com : normalizeText(com,st_,nst_))
    return data

In [14]:
dataset = normalizeData(dataset,standar,nonstandar)

In [15]:
dataset.to_csv("../data/dataset/data_normalizazionprocess_checkpoint.csv",index = False,sep=";")

1. Menghilangkan Angka

In [16]:
def removeNumb(data):
    data['Tweet_Parsed_4'] = data['Tweet_Parsed_4'].str.replace('[^a-zA-Z]+',' ')
    return data

In [17]:
dataset = removeNumb(dataset)

2. Menghilangkan kata tidak bermakna

In [18]:
meaningless = pd.read_csv("../data/stopword/stop.csv", header=None, names=['stop'])
meaningless = meaningless['stop'].tolist()

In [19]:
meaningless

['stop',
 'leaks',
 'sis',
 'guys',
 'boi',
 'dah',
 'btw',
 'lol',
 'koook',
 'the',
 'way',
 'by',
 'pun',
 'bajingan',
 'sumpah',
 'yaampuuun',
 'siiih',
 'shopee',
 'shopeeid',
 'nih',
 'sih',
 'si',
 'ayo',
 'ya',
 'oh',
 'to',
 'yung',
 'hai',
 'lang',
 'nya',
 'haha',
 'ya',
 'nih',
 'shopeedarirumah',
 'tolong',
 'wkwk',
 'kah',
 'dong',
 'doang',
 'hai',
 'hihi',
 'hiks',
 'huft',
 'yang',
 'ya',
 'ni',
 'sialla',
 'nder',
 'wowowoowow',
 'ken',
 'ha',
 'mai',
 'amik',
 'sihh',
 'ko',
 'btsarmy',
 'btsfesta',
 'bts',
 'twt',
 'heu',
 'anjrit',
 'lh',
 'alang',
 'wow',
 'senderr',
 'ekekekeke',
 'sender',
 'atuh',
 'babi',
 'aja',
 'gitu',
 'duh',
 'bro',
 'huhu',
 'hehehe',
 'ahahhahah',
 'haih',
 'superb',
 'ngentot',
 'minn',
 'yah',
 'iphone',
 'pro',
 'max',
 'xs',
 'apple',
 'x',
 'promax',
 'allah']

In [20]:
def removeMeaningless(data,meaningless_):
    for word in meaningless_:
        regex_meaningless = r"\b" + word + r"\b"
        data['Tweet_Parsed_4'] = data['Tweet_Parsed_4'].str.replace(regex_meaningless, '')
    return data

In [21]:
dataset = removeMeaningless(dataset,meaningless)

## Stemming

In [22]:
factory = StemmerFactory()
stemmerID = factory.create_stemmer()

In [23]:
def stemming(text, stemmer_id):
    text_split = text.split(" ")
    stemmed_list = []
    for i in text_split:
        stem_text1 = stemmer_id.stem(i)
#         stem_text2 = stemmer_en.lemmatize(stem_text1, pos="v")
#         stem_text3 = stemmer_en.lemmatize(stem_text2, pos="n")
#         stem_text4 = stemmer_en.lemmatize(stem_text3, pos="a")
        stemmed_list.append(stem_text1)
    stemmed = ' '.join(map(str,stemmed_list))
    return stemmed

def stemData(data, stemmer_id):
    data['Tweet_Parsed_5'] = data['Tweet_Parsed_4']
    data['Tweet_Parsed_5'] = data['Tweet_Parsed_5'].map(lambda com : stemming(com,stemmer_id))
    return data

In [24]:
dataset = stemData(dataset, stemmerID)

In [25]:
dataset.to_csv("../data/dataset/data_stemmingonprocess_checkpoint.csv",index = False,sep=";")

## Stopword Removal

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
stop_words = list(stopwords.words('indonesian')) + list(stopwords.words('english'))
stop_words = stop_words+["rt","retweet","url"]

In [28]:
stop_words[1:10]

['adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir']

In [29]:
len(stop_words)

940

In [30]:
# with open("../data/stopword/stopword_nonltk.pkl","rb") as f:
#     stopword_nonltk = pickle.load(f)

In [31]:
# len(stopword_nonltk)

In [32]:
def removeStopword(data,stop_words_):
    data['Tweet_Parsed_6'] = data['Tweet_Parsed_5']
    for stop_word in stop_words_:
        regex_stopword = r"\b" + stop_word + r"\b"
        data['Tweet_Parsed_6'] = data['Tweet_Parsed_6'].str.replace(regex_stopword, '')
    return data

In [33]:
dataset = removeStopword(dataset,stop_words)

1. Clean space

In [34]:
def clean_space(text):
    text_split = text.split(' ')
    text_clean = [i for i in text_split if i != '']
    text_fix = ' '.join(map(str,text_clean))
    return text_fix

def clean_space_data(data):
    data['Tweet_Parsed_7'] = data['Tweet_Parsed_6']
    data['Tweet_Parsed_7'] = data['Tweet_Parsed_7'].map(lambda com : clean_space(com))
    return data

In [35]:
dataset = clean_space_data(dataset)

In [36]:
dataset['Tweet'].loc[4]

'xs max, senderrr. tapi rugi sih kalo kamu beli xs max baru sekarang.. i mean langsung ke iphone 11 juga gapapa sebenernya secara speknya lebih bagus terussss harganya gak jauh beda.'

In [37]:
dataset['Tweet_Parsed_1'].loc[4]

'xs max, senderrr. tapi rugi sih kalo kamu beli xs max baru sekarang.. i mean langsung ke iphone 11 juga gapapa sebenernya secara speknya lebih bagus terussss harganya gak jauh beda.'

In [38]:
dataset['Tweet_Parsed_2'].loc[4]

'xs max, senderrr. tapi rugi sih kalo kamu beli xs max baru sekarang.. i mean langsung ke iphone 11 juga gapapa sebenernya secara speknya lebih bagus terussss harganya gak jauh beda.'

In [39]:
dataset['Tweet_Parsed_3'].loc[4]

'xs max senderrr tapi rugi sih kalo kamu beli xs max baru sekarang i mean langsung ke iphone 11 juga gapapa sebenernya secara speknya lebih bagus terussss harganya gak jauh beda '

In [40]:
dataset['Tweet_Parsed_4'].loc[4]

'  senderrr tapi rugi  kalau kamu beli   baru sekarang i maksud langsung ke  juga gapapa sebenarnya secara speknya lebih bagus terus harganya tidak jauh beda '

In [41]:
dataset['Tweet_Parsed_5'].loc[4]

'  senderrr tapi rugi  kalau kamu beli   baru sekarang i maksud langsung ke  juga gapapa benar cara speknya lebih bagus terus harga tidak jauh beda '

In [42]:
dataset['Tweet_Parsed_6'].loc[2]

'   kamera  jelek   liat liat     foto  foto  cermin  kamera  '

In [43]:
dataset['Tweet_Parsed_7'].loc[2]

'kamera jelek liat liat foto foto cermin kamera'

In [44]:
def fixData(data, stemming=True, stopword=True):
    if stopword == True:
        data_fix = data.drop(['Tweet_Parsed_1', 'Tweet_Parsed_2','Tweet_Parsed_3','Tweet_Parsed_4','Tweet_Parsed_5','Tweet_Parsed_6'],axis=1)
        data_fix = data_fix.rename(columns={'Tweet_Parsed_7': 'Tweet_Parsed'})
    elif stemming == True:
        data_fix = data.drop(['Tweet_Parsed_1', 'Tweet_Parsed_2','Tweet_Parsed_3','Tweet_Parsed_4','Tweet_Parsed_6','Tweet_Parsed_7'],axis=1)
        data_fix = data_fix.rename(columns={'Tweet_Parsed_5': 'Tweet_Parsed'})
    else:
        data_fix = data.drop(['Tweet_Parsed_1', 'Tweet_Parsed_2','Tweet_Parsed_3','Tweet_Parsed_5','Tweet_Parsed_6','Tweet_Parsed_7'],axis=1)
        data_fix = data_fix.rename(columns={'Tweet_Parsed_4': 'Tweet_Parsed'})
    return data_fix

In [45]:
dataset = fixData(dataset,stemming=False,stopword=True)

In [46]:
def handleEmptyString(data):
    data.loc[data['Tweet_Parsed'] == '', ['Tweet_Parsed']] = " "
    return data

In [47]:
dataset = handleEmptyString(dataset)

In [48]:
dataset.to_csv("../data/dataset/datasetpraproses.csv",index = False,sep=";")

## Postag

In [49]:
dataset = pd.read_csv("../data/dataset/datasetpraproses.csv", sep=';')

In [50]:
dataset

,Tweet,label,Tweet_Parsed
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",baterai negatif,admin normal pakai baterai kapasitas turun
1,Kmera iphone 11 mmeng beda,kamera positif,kamera beda
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera negatif,kamera jelek liat liat foto foto cermin kamera
3,Apple meluncurkan iPhone 11 dengan peningkatan...,baterai positif,luncur tingkat daya tahan baterai sayang laku ...
4,"xs max, senderrr. tapi rugi sih kalo kamu beli...",spesifikasi positif,senderrr rugi beli maksud langsung gapapa spek...
...,...,...,...
743,ga keren banget masa charger bawaannya belum f...,baterai negatif,keren charger bawa cepat charger
744,"yaelah belum fast charging, lama nih kalo ngec...",baterai negatif,yaelah cepat charger charge charger bawa kudu ...
745,butuh waku lama buat ngecharge kalo pake charg...,baterai negatif,butuh waku charge charger bawa daya
746,udah mahal begini masa chargernya blm fast cha...,baterai negatif,udah mahal chargernya cepat charger kecewa


In [51]:
def postag(data):
    dt = []
    for line in data:
        tokens = word_tokenize(str(line)) #Tokenisasi (Membagi kata per kata)
        postag = nltk.pos_tag(tokens)
        dt.append(postag) #memasukan kata yang telah dipostag kedalam list
        print(postag)
    return dt

In [52]:
data_postag = postag(dataset['Tweet_Parsed'])

[('admin', 'NN'), ('normal', 'JJ'), ('pakai', 'NN'), ('baterai', 'NN'), ('kapasitas', 'NNS'), ('turun', 'VBP')]
[('kamera', 'NN'), ('beda', 'NN')]
[('kamera', 'NN'), ('jelek', 'NN'), ('liat', 'NN'), ('liat', 'NN'), ('foto', 'NN'), ('foto', 'NN'), ('cermin', 'NN'), ('kamera', 'NN')]
[('luncur', 'JJ'), ('tingkat', 'NN'), ('daya', 'NN'), ('tahan', 'NN'), ('baterai', 'NN'), ('sayang', 'NN'), ('laku', 'NN'), ('perangkat', 'NN'), ('lunak', 'NN')]
[('senderrr', 'NN'), ('rugi', 'NN'), ('beli', 'NN'), ('maksud', 'NN'), ('langsung', 'NN'), ('gapapa', 'NN'), ('speknya', 'NN'), ('bagus', 'NN'), ('harga', 'NN'), ('beda', 'NN')]
[('bingung', 'NN'), ('pilih', 'NN'), ('review', 'NN'), ('spesifikasi', 'NN'), ('minat', 'NN'), ('bentuk', 'NN'), ('kamera', 'NN'), ('pilih', 'NN'), ('pasang', 'NN'), ('pakai', 'NN'), ('konon', 'NN'), ('sesuai', 'NN')]
[('hasil', 'NN'), ('kamera', 'NN'), ('xiaomi', 'NNP'), ('note', 'NN'), ('adu', 'NN')]
[('tajam', 'NN'), ('layar', 'NN'), ('ultra', 'JJ'), ('wide', 'JJ'), ('ang

[('p', 'NN'), ('kejar', 'NN'), ('raih', 'NN'), ('kualitas', 'NNS'), ('foto', 'VBP'), ('cetak', 'JJ'), ('jelek', 'NN'), ('jalan', 'NN'), ('malas', 'VBP'), ('bawa', 'NN'), ('kamera', 'NN'), ('beneran', 'NN'), ('kamera', 'NN'), ('hpku', 'NN'), ('andal', 'NN')]
[('ios', 'NNS'), ('keluh', 'VBP'), ('baterai', 'JJ'), ('turun', 'JJ'), ('latestnews', 'NNS'), ('os', 'VBP'), ('ios', 'NNS')]
[('charge', 'NN'), ('lambat', 'NN'), ('ampun', 'NN')]
[('luncur', 'JJ'), ('tingkat', 'NN'), ('daya', 'NN'), ('tahan', 'NN'), ('baterai', 'NN'), ('keluh', 'NN'), ('ios', 'NNS'), ('kuras', 'VBP'), ('baterai', 'JJ'), ('ponsel', 'NN')]
[('warna', 'JJ'), ('mint', 'NN'), ('mohon', 'NN'), ('ungu', 'NN')]
[('baterai', 'NN'), ('bener', 'NN'), ('irit', 'NN'), ('perfoma', 'NN')]
[('ken', 'VBN'), ('warna', 'NN'), ('kuning', 'VBG')]
[('alas', 'NNS'), ('haru', 'VBP'), ('beli', 'JJ'), ('karna', 'FW'), ('gaya', 'NN'), ('gaya', 'NN'), ('tunjang', 'NN'), ('kerja', 'VBD'), ('fitur', 'JJ'), ('potret', 'JJ'), ('suka', 'NN'), ('gam

[('harga', 'NN'), ('mahal', 'NN'), ('spesifikasi', 'NN')]
[('milik', 'NNS'), ('memori', 'VBP'), ('internal', 'JJ'), ('memori', 'FW'), ('eksternal', 'JJ')]
[('video', 'NN'), ('tiktok', 'NN'), ('hasil', 'NN'), ('jernih', 'NN')]
[('jalan', 'NN'), ('jalan', 'NN'), ('puas', 'VBP'), ('dapatin', 'NN'), ('foto', 'NN'), ('foto', 'NN'), ('bagus', 'NN')]
[('milik', 'NN'), ('ram', 'NN'), ('gb', 'NN')]
[('sayang', 'NN'), ('ngecas', 'NN')]
[('spesifikasi', 'NN'), ('wajar', 'NN'), ('harga', 'NN'), ('relatif', 'NN')]
[('terkadang', 'NN'), ('kapasitas', 'NNS'), ('baterai', 'VBP'), ('turun', 'JJ'), ('drastis', 'NN'), ('ios', 'NNS')]
[('aplikasi', 'NN'), ('bayar', 'NN')]
[('baterai', 'NN'), ('boros', 'NNS'), ('bawa', 'VBP'), ('cas', 'NN')]
[('kamera', 'NN'), ('kadang', 'NN'), ('suka', 'NN'), ('fokus', 'NN')]
[('cerah', 'NN'), ('kamera', 'NN'), ('kadang', 'NN'), ('suka', 'NN'), ('stabil', 'NN')]
[('komen', 'NNS'), ('kamera', 'VBP'), ('kayak', 'NN'), ('kompor', 'NN'), ('ih', 'NN')]
[('efek', 'NN'), ('glowi

[('harga', 'NN'), ('murah', 'NN'), ('gue', 'NN'), ('beli', 'NN'), ('nyampe', 'NN'), ('juta', 'NN')]
[('kirain', 'NN'), ('harga', 'NN'), ('mahal', 'NN'), ('nyata', 'NN'), ('engga', 'NN'), ('mahal', 'NN')]
[('harga', 'NN'), ('murah', 'NN'), ('ukur', 'NN')]
[('mahal', 'NN'), ('harga', 'NN'), ('sanggup', 'NN'), ('beli', 'NN')]
[('harga', 'NN'), ('mahal', 'RBS'), ('jual', 'JJ'), ('ginjal', 'NN')]
[('sultan', 'JJ'), ('mah', 'NN'), ('mahal', 'NN')]
[('setimapal', 'JJ'), ('gue', 'NN'), ('harga', 'NN'), ('mahal', 'NN'), ('beli', 'JJ'), ('beli', 'JJ'), ('laptop', 'NN')]
[('mahal', 'JJ'), ('harga', 'NN'), ('motor', 'NN')]
[('mahal', 'NN'), ('harga', 'NN'), ('model', 'NN'), ('liat', 'NN'), ('murah', 'NN'), ('seris', 'NN')]
[('ken', 'VBN'), ('suka', 'JJ'), ('liat', 'NN'), ('harga', 'NN')]
[('barang', 'NN'), ('merek', 'NN'), ('mahal', 'NN')]
[('harga', 'NN'), ('masuk', 'NN'), ('akal', 'NN'), ('suka', 'NN')]
[('mahal', 'NN'), ('nulai', 'NN'), ('tukar', 'NN'), ('rupiah', 'NN'), ('dolar', 'JJ'), ('maha

[('spesifikasi', 'NN'), ('lancar', 'NN'), ('jalan', 'NN'), ('usaha', 'JJ'), ('online', 'NN')]
[('spesifikasi', 'NN'), ('nyata', 'NNS'), ('neng', 'NN')]
[('gapernah', 'NN'), ('lot', 'NN'), ('buka', 'IN'), ('tutup', 'NN'), ('aplikasi', 'NN')]
[('gapernah', 'NN'), ('ragu', 'NN'), ('spesifikasi', 'NN')]
[('udah', 'JJ'), ('gabisa', 'NN'), ('headset', 'NN'), ('colok', 'NN'), ('airpods', 'NNS'), ('bluetooth', 'DT')]
[('series', 'NN'), ('series', 'NN'), ('udah', 'JJ'), ('face', 'NN'), ('id', 'NN'), ('suka', 'NN'), ('sidik', 'NN'), ('jari', 'NN')]
[('asa', 'NN'), ('woy', 'NN'), ('ram', 'NN'), ('langsung', 'NN'), ('penuh', 'NN'), ('kaya', 'NN'), ('dikit', 'NN'), ('install', 'NN'), ('aplikasi', 'NN')]
[('aneh', 'NN'), ('udah', 'JJ'), ('tombol', 'NN')]
[('woy', 'NN'), ('lubang', 'NN'), ('headset', 'NN')]
[('gin', 'NN'), ('deh', 'NN'), ('memori', 'FW'), ('cuman', 'JJ'), ('internal', 'JJ'), ('langsung', 'NN'), ('penuh', 'NN')]
[('harga', 'NN'), ('mahal', 'NN'), ('dukung', 'NN'), ('g', 'NN'), ('biar'

[('harga', 'NN'), ('mahal', 'NN'), ('varian', 'JJ'), ('murah', 'NN')]
[('sanggup', 'NN'), ('beli', 'NN'), ('liat', 'NN'), ('ulas', 'NNS')]
[('nabung', 'RB'), ('abad', 'NN'), ('beli', 'NN')]
[('harga', 'NN'), ('sulit', 'NN'), ('terima', 'NN'), ('akal', 'NN')]
[('mahal', 'NN'), ('kualitas', 'NNS'), ('beda', 'VBP')]
[('harga', 'NN'), ('mahal', 'NN'), ('ciut', 'NN'), ('beli', 'NN')]
[('salah', 'NN'), ('lemah', 'NN'), ('harga', 'NN'), ('mahal', 'NN')]
[('coba', 'NN'), ('jual', 'JJ'), ('ginjal', 'NN'), ('boba', 'NN'), ('alias', 'NN')]
[('speknya', 'NN'), ('mantap', 'NN'), ('main', 'JJ'), ('pubg', 'NN'), ('lot', 'NN')]
[('sistem', 'NN'), ('operasi', 'MD'), ('bagus', 'VB')]
[('speknya', 'NN'), ('bagus', 'NN'), ('enak', 'NN'), ('main', 'JJ'), ('game', 'NN'), ('kerja', 'NN')]
[('suka', 'JJ'), ('ios', 'NNS'), ('premium', 'NN'), ('kualitas', 'NNS')]
[('puas', 'NN'), ('sistem', 'NN'), ('operasi', 'JJ'), ('kencang', 'NN'), ('bebas', 'NN'), ('hambat', 'NN')]
[('operasi', 'NN'), ('lancar', 'NN'), ('ba

[('hasil', 'NN'), ('foto', 'NN'), ('bagus', 'NN'), ('dukung', 'NN'), ('estetik', 'VBP'), ('gue', 'NN')]
[('kamera', 'NN'), ('oke', 'VBD'), ('foto', 'JJ'), ('video', 'NN'), ('bagus', 'NN')]
[('semenjak', 'NN'), ('ip', 'NN'), ('suka', 'NN'), ('foto', 'NN'), ('deh', 'NN'), ('kamera', 'NN'), ('bagus', 'NN')]
[('salah', 'NN'), ('beli', 'NN'), ('kamera', 'NN'), ('gue', 'NN'), ('suka', 'NN')]
[('jelek', 'NN'), ('fitur', 'NN'), ('night', 'NN'), ('mode', 'NN'), ('kamera', 'NN'), ('noise', 'NN')]
[('kamera', 'NN'), ('jelek', 'NN'), ('ngezoom', 'NN'), ('pecah', 'NN')]
[('banding', 'VBG'), ('kamera', 'NN'), ('mending', 'VBG'), ('beli', 'JJ'), ('produk', 'NN')]
[('kamera', 'NN'), ('muas', 'NN')]
[('ah', 'NN'), ('kamera', 'NN'), ('banding', 'NN')]
[('kamera', 'NN'), ('bagus', 'NN'), ('moto', 'NN'), ('malam', 'NN'), ('hasil', 'NN'), ('jelek', 'NN')]
[('produk', 'NN'), ('kamera', 'NN'), ('bagus', 'NN'), ('ip', 'NN'), ('banding', 'NN')]
[('kamera', 'NN'), ('foto', 'NN'), ('indoor', 'JJ'), ('cahaya', 'N